
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, fourni par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible.![licence : Creative Commons](images/Licence.jpg) 

Auteurs : C-H. Eyraud, G. Vidal, E. le Jan

------------------------------------------------------------

# Comparaison de  3 mesures avec 3 capteurs identiques

Les instruments de mesure de concentration gazeuses de la famille MQ  fournissent un signal analogique c'est pourquoi il faut les convertir en signal numérique pour pouvoir les exploiter avec une RaspberryPI

## Utilisation de convertisseurs Analogique --> digital

Nous avons à notre disposition deux convertisseurs analogique numérique dont le fonctionnement est identique. Ce qui les diffférencie est la grandeur privilégiée: fréquence d'échantillonnage pour le premier, précision de l'échantillonnage pour le second.

 * [ADS1015](https://iotdk.intel.com/docs/master/upm/classupm_1_1_a_d_s1015.html) Résolution de 12 bit, Fréquence d'échantillonnage Max :3300 (SPS_128 à SPS_3300)
 * [ADS1115](https://iotdk.intel.com/docs/master/upm/classupm_1_1_a_d_s1115.html) Résolution  de 16 bit, Fréquence d'échantillonage Max : 860 (SPS_8 à SPS_860)


Deux modes de fonctionnement sont possibles :
 * en mode différentiel le signal est mesuré par différence entre la broche de sortie et la masse
 * en mode direct le signal est mesuré directement sur la broche de sortie
Nous disposons de 3 capteurs identiques et de 4 broches d'échantillonnage sur le convertisseur, nous ferons deux mesures directes et une mesure par différence. L'exploitation fine de ces éléments requiert le calibrage des capteurs analogiques et le choix des bons paramètres de configuration.

La figure ci-dessous présente l'ensemble du dispositif :

![Convertisseurs vue générale](images/3CapteursTout.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Le câblage du convertisseurs ne présente pas de difficulté particulière, le point requérant un peu d'attention est le choix de l'adresse qui sera envoyée sur le bus `I2C` car il faut choisir cette adresse parmi les 4 possibles :  `0x49` `0x48` `0x4B` `0x4A`. Le choix se fait en établissant une connexion entre la broche `ADDR` et l'une des 4 broches `VDD` `GND` `SCL` `SDA`. Le tableau ci-dessous indique les équivalences des connexions :

| Broche | Adresse |
|:------ | ------- |
| `VDD`  | `0x49`  |
| `GND`  | `0x48`  |
| `SCL`  | `0x4B`  |
| `SDA`  | `0x4A`  |

les 3 capteurs utilisés sont identiques et permettent de mesurer la qualité de l'air 


L'image suivante montre les connexions au convertisseur analogique numérique ADS 1015 :

![Vue des connexions au convertisseur](images/3CapteursADS1015.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)

L'image suivante montre les connexions au convertisseur de niveau 5V-3,3V (Level Shifter) :

![Le level shifter](images/3CapteursLevelShifter.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Importation des librairies nécessaires à l'utilisation du convertisseur. On n'importe aucune librairie propre aux capteurs de gaz car il s'agit de capteurs `analogiques` qui transmettent une variation de potentiel sur un fil série.

In [1]:
import mraa, upm
from time import sleep
from upm import pyupm_ads1x15 as upm

## Récupération des paramètres d'état de l'échantillonneur

On déclare ici l'échantillonneur et on récupère les valeurs de ses paramètres d'opération (gain, fréquence et mode). 
 * getGain()
 * getSPS()
 * getContinuous()
 
On fixe ensuite ces paramètres pour la manipulation que l'on souhaite effectuer. Le gain par défaut est 1 (GAIN_ONE) mais on peut le fixer à 2/3 (GAIN_TWOTHIRDS), 2 (GAIN_TWO),4 (GAIN_FOUR),8 (GAIN_EIGHT),16 (GAIN_SIXTEEN)
 * setGain()
 * setSPS()
 * setContinuous()

In [17]:
gaz = upm.ADS1015(0, 0x49)
gain = gaz.getGain()
sps = gaz.getSPS()
cont = gaz.getContinuous() 
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), 
       "Continuous : {0:5d}".format(cont))
gaz.setContinuous(True) # Pour ne pas faire de conversion en continu
gaz.setGain(upm.ADS1X15.GAIN_ONE) # valeur par défaut de 1 pour le gain
gaz.setSPS(upm.ADS1015.SPS_250) # Passage à un taux d'échantillonage de 250 Sample Per Second
# Vérification
gain = gaz.getGain()
sps = gaz.getSPS()
cont = gaz.getContinuous() 
print ("\nNouvelles valeurs :")
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), 
       "Continuous : {0:5d}".format(cont))

Gain :   512  SPS :    32 Continuous :     1

Nouvelles valeurs :
Gain :   512  SPS :    32 Continuous :     1


## Mesure directe des variables qualité de l'air

La première mesure est faite par différence, les deux dernières directement.

In [16]:
air1 = gaz.getSample(upm.ADS1X15.DIFF_0_1)
air2 = gaz.getSample(upm.ADS1X15.SINGLE_2)
air3 = gaz.getSample(upm.ADS1X15.SINGLE_3)
print ("Qualité Air 1 : {0:.2f}\n".format(air1),
       "Qualité Air 2 : {0:.2f}\n".format(air2),
       "Qualité Air 3 : {0:.2f}\n".format(air3))

Qualité Air 1 : 0.26
 Qualité Air 2 : 0.26
 Qualité Air 3 : 0.26



# Série de mesures à l'aide d'une boucle
Dans le groupe suivant on répète 20 fois la mesure en partant de 0 et en allant jusqu'à 19 (inférieur à 20) au pas de 1 par défaut.

In [14]:
for num in range(0,20):
    air1 = gaz.getSample(upm.ADS1X15.DIFF_0_1)
    air2 = gaz.getSample(upm.ADS1X15.SINGLE_2)
    air3 = gaz.getSample(upm.ADS1X15.SINGLE_3)
    print ("Qualité Air 1 : {0:.2f}\n".format(air1),
       "Qualité Air 2 : {0:.2f}\n".format(air2),
       "Qualité Air 3 : {0:.2f}\n".format(air3))    
    sleep(2)

Qualité Air 1 : 1.50
 Qualité Air 2 : 1.13
 Qualité Air 3 : 1.14

Qualité Air 1 : 1.53
 Qualité Air 2 : 1.12
 Qualité Air 3 : 1.11

Qualité Air 1 : 1.50
 Qualité Air 2 : 1.99
 Qualité Air 3 : 1.11

Qualité Air 1 : 1.52
 Qualité Air 2 : 1.95
 Qualité Air 3 : 1.11

Qualité Air 1 : 1.51
 Qualité Air 2 : 1.99
 Qualité Air 3 : 1.14

Qualité Air 1 : 1.51
 Qualité Air 2 : 1.94
 Qualité Air 3 : 1.11

Qualité Air 1 : 1.98
 Qualité Air 2 : 2.00
 Qualité Air 3 : 1.13

Qualité Air 1 : 1.52
 Qualité Air 2 : 1.13
 Qualité Air 3 : 1.12

Qualité Air 1 : 1.50
 Qualité Air 2 : 1.96
 Qualité Air 3 : 1.15

Qualité Air 1 : 1.49
 Qualité Air 2 : 1.13
 Qualité Air 3 : 1.15

Qualité Air 1 : 1.50
 Qualité Air 2 : 1.96
 Qualité Air 3 : 1.54

Qualité Air 1 : 1.53
 Qualité Air 2 : 1.95
 Qualité Air 3 : 1.12

Qualité Air 1 : 1.50
 Qualité Air 2 : 1.94
 Qualité Air 3 : 1.13

Qualité Air 1 : 1.50
 Qualité Air 2 : 1.96
 Qualité Air 3 : 1.15

Qualité Air 1 : 1.53
 Qualité Air 2 : 1.99
 Qualité Air 3 : 1.12

Qualité Ai